In [2]:
import numpy as np
from scipy.optimize import fsolve
from sklearn.linear_model import LogisticRegression
import math


In [ ]:
# Probability of a point being white (k=0) if x1 >= 0
P_WHITE_IF_X1_GE_0 = 0.7  # denoted as 'p' in the problem description
# Probability of a point being white (k=0) if x1 < 0
P_WHITE_IF_X1_LT_0 = 0.3  # denoted as 'q' in the problem description
# Probability that the VC bound does NOT hold (epsilon-net probability)
ETA = 0.05                # denoted as 'η'
# Maximum allowed absolute difference between theoretical and empirical risks
EPSILON = 0.1             # denoted as 'ε'
# Size of the new sample for testing the chosen strategy
N_TEST_SAMPLE = 1000      # denoted as 'N'


In [ ]:

# Task 1: Calculate Vapnik-Chervonenkis (VC) Dimension
def calculate_vc_dimension():
    """
    Calculates the Vapnik-Chervonenkis (VC) dimension for the given set of strategies.
    The strategies are linear classifiers in R^2, defined by (x.a) >= theta or (x.a) < theta.
    For linear classifiers in a d-dimensional space, the VC dimension is d+1.
    Here, the input space is R^2, so d=2.
    """
    vc_dim = 2 + 1
    return vc_dim


In [ ]:

# Task 2: Estimate Training Sample Size (l)
def estimate_sample_size(vc_dim, epsilon, eta):
    """
    Estimates the minimum training sample size 'l' required such that with probability
    1 - eta, the absolute difference (modulo) between theoretical and empirical risks
    for all admissible strategies does not exceed epsilon.

    This estimation uses the Vapnik-Chervonenkis bound:
    epsilon >= sqrt( (8/l) * ( VC_dim * ln(2l/VC_dim) + ln(4/eta) ) )

    Since this is a transcendental equation for 'l', it is solved numerically using fsolve.
    """
    # Define the function for which we want to find the root (f(l) = 0)
    def vc_bound_equation(l_val):
        # Ensure l_val and vc_dim are positive to avoid math domain errors (log of non-positive)
        if l_val <= 0 or vc_dim <= 0:
            # Return a large positive value to push the solver towards positive l
            return float('inf')
        
        # Calculate the terms inside the square root
        term_vc_log = vc_dim * math.log(2 * l_val / vc_dim)
        term_eta_log = math.log(4 / eta)
        
        # The equation we want to solve for l: sqrt(...) - epsilon = 0
        return math.sqrt((8 / l_val) * (term_vc_log + term_eta_log)) - epsilon

    # Provide an initial guess for 'l'. A heuristic based on simpler bounds (l ~ VC_dim / epsilon^2)
    # can help fsolve converge. We multiply by 10 to give a slightly larger starting point.
    initial_guess = vc_dim / (epsilon**2) * 10 
    
    # Use fsolve to find the root. fsolve returns an array, so we take the first element.
    l_estimated = fsolve(vc_bound_equation, initial_guess)[0]

    # The sample size must be a positive integer. We take the ceiling and ensure it's at least 1.
    return math.ceil(max(1, l_estimated))


In [8]:

# --- Task 3: Generate Sample, Choose Strategy, Test Strategy ---

def generate_data(num_samples, p_white_x1_ge_0, p_white_x1_lt_0):
    """
    Generates a dataset of 2D points (x1, x2) and their corresponding labels.
    Points are generated from a standard normal distribution N((0,0), I).
    Labels are assigned based on x1 and the probabilities p and q.
    Label k=0 for white, k=1 for black.
    """
    # Generate points from a 2D standard normal distribution
    points = np.random.multivariate_normal([0, 0], [[1, 0], [0, 1]], num_samples)
    labels = np.zeros(num_samples, dtype=int) # Initialize labels array

    for i in range(num_samples):
        x1 = points[i, 0]
        
        # Assign label based on x1 value and probabilities p or q
        if x1 >= 0:
            # If x1 >= 0, label is white (0) with probability p, black (1) with probability 1-p
            if np.random.rand() < p_white_x1_ge_0:
                labels[i] = 0  # White
            else:
                labels[i] = 1  # Black
        else:
            # If x1 < 0, label is white (0) with probability q, black (1) with probability 1-q
            if np.random.rand() < p_white_x1_lt_0:
                labels[i] = 0  # White
            else:
                labels[i] = 1  # Black

    return points, labels

def main():
  
    print("--- Recognition by Training Sample ---")

    # --- Task 1 Execution ---
    print("\n--- Task 1: VC Dimension Calculation ---")
    vc_dimension = calculate_vc_dimension()
    print(f"The Vapnik-Chervonenkis (VC) dimension of the given set of strategies is: {vc_dimension}")

    # --- Task 2 Execution ---
    print("\n--- Task 2: Training Sample Size Estimation ---")
    # Validate ETA and EPSILON parameters
    if not (0 < ETA < 1 and 0 < EPSILON < 1):
        print("Error: ETA and EPSILON must be strictly between 0 and 1. Please adjust parameters.")


    training_sample_size = estimate_sample_size(vc_dimension, EPSILON, ETA)
    print(f"To ensure that with probability 1 - {ETA}, the absolute difference between theoretical")
    print(f"and empirical risks does not exceed {EPSILON}, the estimated minimum training sample size (l) is: {training_sample_size}")
    print("(Note: This is a numerical estimate based on the VC bound, and may result in a large 'l'.)")


    # --- Task 3 Execution ---
    print("\n--- Task 3: Strategy Generation and Testing ---")
    print(f"Current Parameters for Task 3:")
    print(f"  p (P_WHITE_IF_X1_GE_0) = {P_WHITE_IF_X1_GE_0}")
    print(f"  q (P_WHITE_IF_X1_LT_0) = {P_WHITE_IF_X1_LT_0}")
    print(f"  N (test sample size) = {N_TEST_SAMPLE}")

    # 3.1 Generate training sample
    print(f"\nGenerating training sample of size {training_sample_size}...")
    X_train, y_train = generate_data(training_sample_size, P_WHITE_IF_X1_GE_0, P_WHITE_IF_X1_LT_0)
    print("Training sample generated.")
    
    print("Training Logistic Regression model to find optimal strategy...")
    model = LogisticRegression(solver='liblinear', random_state=42) # liblinear is good for small datasets
    model.fit(X_train, y_train)

    optimal_a = model.coef_[0]
    optimal_theta = -model.intercept_[0]

    print(f"Optimal strategy found: a = {optimal_a}, theta = {optimal_theta:.4f}")

    # Define the chosen strategy function using the derived 'a' and 'theta'
    def chosen_strategy(x_point, a_vec, theta_val):
        """Applies the chosen linear classification strategy to a single point."""
        return 0 if np.dot(x_point, a_vec) >= theta_val else 1

    # Calculate empirical risk on the training sample using the chosen strategy
    train_predictions = np.array([chosen_strategy(x, optimal_a, optimal_theta) for x in X_train])
    empirical_risk_train = np.mean(train_predictions != y_train)
    print(f"Empirical risk on training sample: {empirical_risk_train:.4f}")

    # 3.3 Test the chosen strategy on a new sample
    print(f"\nGenerating new test sample of size {N_TEST_SAMPLE}...")
    X_test, y_test = generate_data(N_TEST_SAMPLE, P_WHITE_IF_X1_GE_0, P_WHITE_IF_X1_LT_0)
    print("Test sample generated.")

    test_predictions = np.array([chosen_strategy(x, optimal_a, optimal_theta) for x in X_test])
    empirical_risk_test = np.mean(test_predictions != y_test)
    print(f"Empirical risk on test sample: {empirical_risk_test:.4f}")


In [9]:

# Entry point for the script execution
if __name__ == "__main__":
    main()


--- Recognition by Training Sample ---

--- Task 1: VC Dimension Calculation ---
The Vapnik-Chervonenkis (VC) dimension of the given set of strategies is: 3

--- Task 2: Training Sample Size Estimation ---
To ensure that with probability 1 - 0.05, the absolute difference between theoretical
and empirical risks does not exceed 0.1, the estimated minimum training sample size (l) is: 27024
(Note: This is a numerical estimate based on the VC bound, and may result in a large 'l'.)

--- Task 3: Strategy Generation and Testing ---
Current Parameters for Task 3:
  p (P_WHITE_IF_X1_GE_0) = 0.7
  q (P_WHITE_IF_X1_LT_0) = 0.3
  N (test sample size) = 1000

Generating training sample of size 27024...
Training sample generated.
Training Logistic Regression model to find optimal strategy...
Optimal strategy found: a = [-0.70959334  0.0139563 ], theta = -0.0182
Empirical risk on training sample: 0.6991

Generating new test sample of size 1000...
Test sample generated.
Empirical risk on test sample: 0

C:\Users\OlesiaBrusentseva\AppData\Local\Temp\ipykernel_12060\2455665693.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  term_vc_log = vc_dim * math.log(2 * l_val / vc_dim)
C:\Users\OlesiaBrusentseva\AppData\Local\Temp\ipykernel_12060\2455665693.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.sqrt((8 / l_val) * (term_vc_log + term_eta_log)) - epsilon
